In [1]:
import pandas as pd
import numpy as np
import re

from datetime import datetime
from dateutil.relativedelta import relativedelta

In [2]:
dataset = pd.read_csv('train/train.csv')

In [3]:
dataset.shape

(233154, 41)

In [4]:
dataset.head()

,UniqueID,disbursed_amount,asset_cost,ltv,branch_id,supplier_id,manufacturer_id,Current_pincode_ID,Date.of.Birth,Employment.Type,...,SEC.SANCTIONED.AMOUNT,SEC.DISBURSED.AMOUNT,PRIMARY.INSTAL.AMT,SEC.INSTAL.AMT,NEW.ACCTS.IN.LAST.SIX.MONTHS,DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS,AVERAGE.ACCT.AGE,CREDIT.HISTORY.LENGTH,NO.OF_INQUIRIES,loan_default
0,420825,50578,58400,89.55,67,22807,45,1441,01-01-84,Salaried,...,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0,0
1,537409,47145,65550,73.23,67,22807,45,1502,31-07-85,Self employed,...,0,0,1991,0,0,1,1yrs 11mon,1yrs 11mon,0,1
2,417566,53278,61360,89.63,67,22807,45,1497,24-08-85,Self employed,...,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0,0
3,624493,57513,66113,88.48,67,22807,45,1501,30-12-93,Self employed,...,0,0,31,0,0,0,0yrs 8mon,1yrs 3mon,1,1
4,539055,52378,60300,88.39,67,22807,45,1495,09-12-77,Self employed,...,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,1,1


Checking if there are any null values in the dataset

In [5]:
dataset.isnull().sum()

UniqueID                                  0
disbursed_amount                          0
asset_cost                                0
ltv                                       0
branch_id                                 0
supplier_id                               0
manufacturer_id                           0
Current_pincode_ID                        0
Date.of.Birth                             0
Employment.Type                        7661
DisbursalDate                             0
State_ID                                  0
Employee_code_ID                          0
MobileNo_Avl_Flag                         0
Aadhar_flag                               0
PAN_flag                                  0
VoterID_flag                              0
Driving_flag                              0
Passport_flag                             0
PERFORM_CNS.SCORE                         0
PERFORM_CNS.SCORE.DESCRIPTION             0
PRI.NO.OF.ACCTS                           0
PRI.ACTIVE.ACCTS                

* We will convert date of birth to age
* Converting AVERAGE.ACCT.AGE and CREDIT.HISTORY.LENGTH to number format

Converting the average account age to years in decimal format

In [6]:
dataset['AVERAGE.ACCT.AGE'] = dataset['AVERAGE.ACCT.AGE'].map(lambda x: int(re.findall('\d+', x)[0])*12 +int(re.findall('\d+', x)[1]))

Converting the credit history length to years in decimal format

In [7]:
dataset['CREDIT.HISTORY.LENGTH'] = dataset['CREDIT.HISTORY.LENGTH'].map(lambda x: int(re.findall('\d+', x)[0])*12 + int(re.findall('\d+', x)[1]))

Converting the dates in Disbursal date column and date of birth column to datetime format

In [8]:
dataset['DisbursalDate'] = dataset['DisbursalDate'].map(lambda x: datetime.strptime(x, '%d-%m-%y'))

In [9]:
dataset['Date.of.Birth'] = dataset['Date.of.Birth'].map(lambda x: datetime.strptime(x, '%d-%m-%y'))

In [10]:
dataset['Date.of.Birth'] = dataset['Date.of.Birth'].map(lambda x: x - relativedelta(years = 100) if x > datetime.now() else x)

Creating a new column age at disbursal

In [11]:
dataset['AGE.AT.DISBURSAL'] = dataset['DisbursalDate'] - dataset['Date.of.Birth']

In [12]:
dataset['AGE.AT.DISBURSAL'] = dataset['AGE.AT.DISBURSAL'].map(lambda x: np.round(x.days/365, 2))

## Checking the correlation 

In [13]:
pd.DataFrame(dataset.corr()["loan_default"].sort_values(ascending=False))

,loan_default
loan_default,1.000000
ltv,0.098208
disbursed_amount,0.077675
State_ID,0.048075
VoterID_flag,0.043747
NO.OF_INQUIRIES,0.043678
PRI.OVERDUE.ACCTS,0.040872
DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS,0.034462
UniqueID,0.033848
branch_id,0.030193


In [14]:
sum([1 for i in dataset['loan_default'][dataset['Employment.Type'].notnull() == False] if i == 1])

1644

In [15]:
sum([1 for i in dataset['loan_default'][dataset['Employment.Type'].notnull() == False] if i == 0])

6017

In [16]:
sum([1 for i in dataset['loan_default'] if i == 1])

50611

In [17]:
sum([1 for i in dataset['loan_default'] if i == 0])

182543

In [18]:
6017/182543

0.03296209660189654

In [19]:
1644/50611

0.03248305704293533

Since both 1's and 0's are equal percentage in the null values, we will go ahead and drop the null rows

In [22]:
dataset = dataset.dropna()

In [25]:
dataset = dataset.reset_index(drop = True)

In [27]:
dataset['disbursed_amount'].corr(dataset['asset_cost'])

0.7460367785814574

In [35]:
len(set(dataset['State_ID']))

22

Dropping the id columns and less important columns

In [36]:
unimp_cols = ['UniqueID', 'branch_id', 'supplier_id', 'Current_pincode_ID', 'Date.of.Birth', 'Employment.Type', 
              'DisbursalDate', 'Employee_code_ID', 'State_ID']
dataset.drop(unimp_cols, axis = 1, inplace = True)

In [37]:
dataset.isnull().sum()

disbursed_amount                       0
asset_cost                             0
ltv                                    0
manufacturer_id                        0
MobileNo_Avl_Flag                      0
Aadhar_flag                            0
PAN_flag                               0
VoterID_flag                           0
Driving_flag                           0
Passport_flag                          0
PERFORM_CNS.SCORE                      0
PERFORM_CNS.SCORE.DESCRIPTION          0
PRI.NO.OF.ACCTS                        0
PRI.ACTIVE.ACCTS                       0
PRI.OVERDUE.ACCTS                      0
PRI.CURRENT.BALANCE                    0
PRI.SANCTIONED.AMOUNT                  0
PRI.DISBURSED.AMOUNT                   0
SEC.NO.OF.ACCTS                        0
SEC.ACTIVE.ACCTS                       0
SEC.OVERDUE.ACCTS                      0
SEC.CURRENT.BALANCE                    0
SEC.SANCTIONED.AMOUNT                  0
SEC.DISBURSED.AMOUNT                   0
PRIMARY.INSTAL.A

## Converting categorical variables to onehot

* manufacturer_id
* PERFORM_CNS.SCORE.DESCRIPTION 